In [23]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import astropy.constants as c 
import astropy.units as u

import math

In [41]:
KB = c.k_B.cgs
PI = np.pi
E_B = (4.52 * u.eV).cgs
HBAR = c.hbar.cgs
W_VIB = 5.4 * 10 ** 4 * u.Hz
MOI = 9.1 * 10 ** (-41) * u.g * (1 * u.cm) ** (-3)
I = (13.6 * u.eV).cgs
A0 = c.a0.cgs
M_H = 1.67 * 10 ** (-24) * u.g
H = c.h.cgs

In [4]:
#H2 internal partition functions

def h2_elec(T):
    e = -E_B/(KB * T)
    return 1 + np.exp(e.value)

def rot_deg(T, j):
    e = (-j * (j + 1) * HBAR ** 2 / MOI) / (KB * T)
    return (2*j + 1) * np.exp(e.value)

def vib_f(T, n):
    e = -(n+0.5)*HBAR*W_VIB/(KB*T)
    return np.exp(e.value)

def parallel(T):
    z_para = 0
    z_ortho = 0
    z_vib = 0

    for i in range(0, 100):
        
        z_vib = z_vib + vib_f(T, i)

        if i%2 == 0:
            z_para = z_para + rot_deg(T, i)
            
        if i%2 != 0:
            z_ortho = z_ortho + rot_deg(T, i)
            
    return z_vib, z_para, 3*z_ortho

def h2_int(T):
    h2_vib, h2_pararot, h2_orthorot = parallel(T)
    
    return h2_elec(T) * (h2_pararot+ h2_orthorot) * h2_vib

In [5]:
#HI internal partition function

def n_max(T, P):
    return np.sqrt((KB * T) ** (1/3) / (2 * A0 * P ** (1/3)))

def h1_exp(T, n):
    e = -I * (1 - 1/(n**2))/(KB * T)
    return n ** 2 * np.exp(e.value)

def h1_elec(T, P):
    n_m = n_max(T, P)
    z_sum = 0
    
    for n in range(1, math.floor(n_m.value)):
        z_sum = z_sum + h1_exp(T, n)

    return 4 * z_sum
            

In [39]:
def C(T, P):
    
    z_tot = h1_elec(T, P) ** 2 / h2_int(T)

    e = -E_B / (KB * T)
    
    return ((T * KB) ** (5/2) / P) * (PI / H) ** (3/2) * M_H ** (3/2) * z_tot * np.exp(e.value)

def ion_frac(T, P):
    return (-C(T,P).value + np.sqrt(C(T,P).value ** 2 + 4 * C(T,P).value))/2